# Demo for LumiSpy package working with AttoLight SEM CL data

This notebook shows:

- Loading a `HYPCard.sur` file correctly with the lumispy load function
- Plotting cathodoluminescence data in different ways
- Dealing with metadata
- Correcting for spectral defects

Import packages:

In [6]:
%matplotlib qt
#For pop-up window plots, with interactive functionality. If error, use instead %matplotlib tk
import lumispy as lum
import hyperspy.api as hs
import matplotlib.pyplot as plt
import os

## Loading HYPCard files

Use the `hs.load()` function with the `signal_type` class for AttoLight CL-SEM specific data (`CL_SEM`).
State the relative path to the `HYPCard.bin` file.

*You can also leave the path empty. A pop-up window will appear to select the `HYPCard.sur` file from the browser.*

In [82]:
root_folder = "demo-files/load_from_AttoLightSEM/"
import glob
path = os.path.join(root_folder, 'HYPCard.sur')

cl = hs.load(path, signal_type='CL_SEM',)
cl

<Signal1D, title: , dimensions: (64, 64|1024)>

In [8]:
cl.plot()

In [83]:
cl = lum.signals.CLSEMSpectrum(cl)
cl.axes_manager.signal_axes[0].offset = 426.7578430175781
cl.axes_manager.signal_axes[0].scale = 0.5366604924201965
cl.axes_manager.signal_axes[0].units = 'nm'

## Metadata

The `.sur` file contains the general Hyperspy-specific metadata (`cl.metadata`).
A more detailed metadata is also available via `cl.original_metadata`:

In [84]:
#cl.metadata

In [85]:
cl.original_metadata.Object_0_Channel_0.Parsed

AttributeError: 'DictionaryTreeBrowser' object has no attribute 'Object_0_Channel_0'

## Data pre-processing

Let's start some pre-processing methods:

### Background removal

In the AttoLight system, if a background is taken before mapping, the background is stored automatically in the signal folder.

Otherwise, manually load a background file, using `np.loadtxt(path_to_file)`.

In [86]:
try:
    bkg_path = os.path.join(root_folder, 'Background*.txt')
    bkg_path = glob.glob(bkg_path)[0]
except IndexError:
    print('Please, specify a `background.txt` file path below:')
    bkg_path = ''

In [87]:
import numpy as np
cl_bkg = np.loadtxt(bkg_path)[1]
print(cl_bkg.shape)

# Run background subtraction
cl = cl - cl_bkg

(1024,)


### Correction of acquisition defects

Correct for the intrinsic shift caused by the misalignment of the grating with the spectrometer aperture centre.

In [48]:
cl.original_metadata.Object_0_Channel_0.Parsed

├── CCD
│   ├── Baseline Clamp = Off
│   ├── Camera Model = A.920
│   ├── Capture Depth = 1024
│   ├── Exposure Time = 0.3
│   ├── Exposure Time_units = s
│   ├── Horizontal Binning = 1
│   ├── Pixel Width = 0.026
│   ├── Pixel Width_units = mm
│   ├── ReadMode = FVB
│   ├── Readout Rate (horizontal pixel shift) = 1Mhz
│   ├── Signal Amplification = x1
│   ├── Speed (vertical pixel shift) = 3.3
│   ├── Temperature = -69
│   ├── Temperature_units = °C
│   └── Voltage Amplitude (vertical pixel shift) = normal
├── CHANNELS
│   ├── Channel1 = SE
│   └── Channel2 = CCD
├── SCAN
│   ├── HYP Dwelltime = 300.000012
│   ├── HYP Dwelltime_units = ms
│   ├── Mode = HYP
│   ├── Reference_Size_X = 512
│   ├── Reference_Size_Y = 512
│   ├── Resolution_X = 64
│   ├── Resolution_X_units = pixels
│   ├── Resolution_Y = 64
│   ├── Resolution_Y_units = pixels
│   ├── Rotate = 0.0
│   ├── Rotate_units = deg
│   ├── Size_X = 1024
│   ├── Size_Y = 1024
│   ├── Start_X = 0
│   ├── Start_Y = 0
│   ├── Voltage Calibration Range_X = 9.2
│   ├── Voltage Calibration Range_X_units = V
│   ├── Voltage Calibration Range_Y = 9.2
│   └── Voltage Calibration Range_Y_units = V
├── SEM
│   ├── Aperture = 100
│   ├── Aperture Chamber Pressure = 1.0426e-07
│   ├── Aperture Chamber Pressure_units = Torr
│   ├── Aperture_units = um
│   ├── Astigmatism 1 X = -0.01
│   ├── Astigmatism 1 X_units = V
│   ├── Astigmatism 1 Y = 0.0
│   ├── Astigmatism 1 Y_units = V
│   ├── Astigmatism 2 X = 17.97
│   ├── Astigmatism 2 X_units = V
│   ├── Astigmatism 2 Y = -6.64
│   ├── Astigmatism 2 Y_units = V
│   ├── Attomic ReadedMagnification = 22000.0
│   ├── Attomic zoom = 40
│   ├── Beam Current = 0.0
│   ├── Beam Current_units = uA
│   ├── Beam Energy = 5998.0
│   ├── Beam Energy_units = V
│   ├── Blanker = Off
│   ├── Blanker_units = um
│   ├── Cathode Heating Current = 1.44
│   ├── Cathode Heating Current_units = A
│   ├── Cathode Heating Voltage = 0.8
│   ├── Cathode Heating Voltage_units = V
│   ├── Deflector 1 X = 0.0
│   ├── Deflector 1 X_units = V
│   ├── Deflector 1 Y = 0.0
│   ├── Deflector 1 Y_units = V
│   ├── Deflector 2 X = -0.21
│   ├── Deflector 2 X_units = V
│   ├── Deflector 2 Y = -0.15
│   ├── Deflector 2 Y_units = V
│   ├── Emission Current = 0.0
│   ├── Emission Current_units = uA
│   ├── Emission Status = ON
│   ├── Extractor Voltage = 4749.0
│   ├── Extractor Voltage_units = V
│   ├── Gun Chamber Pressure = 3.8028e-10
│   ├── Gun Chamber Pressure_units = Torr
│   ├── Gun Lens = 1.2
│   ├── Gun Lens_units = A
│   ├── Isolation Valve = Opened
│   ├── Objective Lens = 0.325544
│   ├── Objective Lens_units = A
│   ├── Real Magnification = 26391.201172
│   ├── Scintillator Voltage = 6010.0
│   ├── Scintillator Voltage_units = V
│   ├── Suppressor Bias = 1004.0
│   └── Suppressor Bias_units = V
├── SITE IMAGE
│   ├── Dwell Time = 300000.011921
│   ├── Dwell Time_units = us
│   ├── Field of view = 0.005
│   ├── Field of view_units = um
│   ├── Number of pixels (x axis) = 64
│   ├── Number of pixels (x axis)_units = px
│   ├── Number of pixels (y axis) = 64
│   ├── Number of pixels (y axis)_units = px
│   ├── stage_model = 10010
│   ├── stage_position_x = -4.600102
│   ├── stage_position_y = -0.743008
│   ├── stage_position_z = -1.368159
│   ├── stage_rotation_x = -0.0
│   ├── stage_rotation_y = -0.0
│   └── stage_rotation_z = -0.0
├── SPECTROMETER
│   ├── Central wavelength = 700.01
│   ├── Central wavelength_units = nm
│   ├── Entrance slit width = 7000
│   ├── Entrance slit width_units = um
│   ├── Exit mirror position = lateral
│   ├── Exit slit width = 0
│   ├── Exit slit width_units = um
│   ├── Exit spectral resolution = 0
│   ├── Exit spectral resolution_units = nm
│   ├── Filter = None
│   ├── Grating = 150/500
│   ├── Grating - Blaze Angle = 0
│   ├── Grating - Groove Density = 150
│   ├── Grating - Groove Density_units = gr/mm
│   ├── Turret = IHR
│   ├── Turret id = 0
│   └── Turret_units = 320
└── WAFER
    ├── Center Position X = 0.0
    ├── Center Posi

In [67]:
calibration_factor = 131072
grating = int(cl.original_metadata.Object_0_Channel_0.Parsed.SPECTROMETER.Grating__Groove_Density)

if grating == 150:
    correction_factor_grating = 2.73E-04 # 150 gr/mm grating
elif grating == 600:
    correction_factor_grating = 6.693659836087227e-05 # 600 gr/mm grating
else:
    raise ImportError('Grating correction not available')

fov = cl.original_metadata.Object_0_Channel_0.Parsed.SITE_IMAGE.Field_of_view

grating_calibrations = {
    'cal_factor' : calibration_factor,
    'corr_factor_grating' : correction_factor_grating,
    'field_of_view_um' : fov,
}

{'cal_factor': 131072,
 'corr_factor_grating': 0.000273,
 'field_of_view_um': 0.005}

In [74]:
cl.correct_grating_shift(*grating_calibrations.values())

ValueError: The value is out of the axis limits

The edges of the scan get higher intensities, so they can be cropped. 
If you set the `inplace` parameter to `True` the original CLSEMSpectrum object will be modified, if `False` a cropped copy of it will be created.

In [88]:
cl = cl.crop_edges(crop_px=5)

There are also corrections for the cosmic rays (pixels with sharp spikes):

In [95]:
cl_clean = cl.remove_spikes()

C:\Users\jf631\Documents\GitHub\jordiferrero\lumispy\lumispy\signals\cl_spectrum.py:91: UserWarning: Threshold value: 24.00
  warn('Threshold value: {:.2f}'.format(spikes_removal.threshold), UserWarning)


In [100]:
# If not all spikes are removed, you can use the GUI to manually select pixels:
cl_clean.remove_spikes(interactive=True, inplace=True)

AttributeError: 'NoneType' object has no attribute 'threshold'

## Plotting data

Plot the corrected data:

In [101]:
cl_clean.plot()

### Panchromatic image:

In [106]:
cl_clean.T.mean().plot(cmap='viridis')

Plot the average CL spectrum:

In [107]:
cl_clean.mean().plot()

## Work in eV units

In [108]:
cl_clean.to_eV()

ERROR:traits:Exception occurred in traits notification handler for object: <Axes manager, axes: (54, 54|)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
     <undefined> |     54 |      1 |       6 |       1 | <undefined> 
     <undefined> |     54 |     10 |       6 |       1 | <undefined> 
---------------- | ------ | ------ | ------- | ------- | ------ , trait: _axes_items, old value: <undefined>, new value: TraitListEvent(index=2, removed=[<Unnamed axis, size: 1024>], added=[])
Traceback (most recent call last):
  File "C:\Users\jf631\.conda\envs\dev\lib\site-packages\traits\trait_notifiers.py", line 522, in _dispatch_change_event
    self.dispatch(handler, *args)
  File "C:\Users\jf631\.conda\envs\dev\lib\site-packages\traits\trait_notifiers.py", line 484, in dispatch
    handler(*args)
  File "C:\Users\jf631\Documents\GitHub\jordiferrero\hyperspy\hyperspy\axes.py", line 1568, in _on_size_changed
  

AttributeError: 'UniformDataAxis' object has no attribute 'real'

In [109]:
cl_clean.plot()

ValueError: axes don't match array

## Post-processing

Look for other `lumispy-demos` notebooks to find examples on fitting luminescence data.


